# IMT 575

## Data Science 3 Project

## Aditya Wakade, Manasi Kulkarni, Prem Shah

### Prediction Stack Overflow Question Tags using doc2vec

This script is used for cleaning, transforming and slicing the dataset.

In [4]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
import itertools
import operator
from collections import Counter
from nltk import pos_tag
from textblob import TextBlob
from nltk.corpus import stopwords
from sklearn.cross_validation import train_test_split
from scipy import sparse as sp_sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from collections import defaultdict

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [19]:
#Import data

ques = pd.read_csv('./Questions.csv', encoding = 'latin-1')
tags = pd.read_csv('./Tags.csv')

In [6]:
top_10 = pd.read_pickle('top10')

In [20]:
#Subset of questions that have score more than 0

ques = ques[ques.Score > 0]

In [97]:
#Save the subset file 

ques.to_csv('./cropped.csv')

In [98]:
#Read the sample file

ques = pd.read_csv('./cropped.csv', encoding = 'latin-1')

In [21]:
#Check length

len(ques)

594057

## Data Cleaning

In [22]:
tags.head()

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


In [17]:
tags_unique = tags[tags['Id'].isin(ques['Id'])]

In [31]:
len(tags_unique['Tag'].value_counts().reset_index())

32058

In [83]:
# Add necessary stopwords

stopwords1 =  set(stopwords.words('english'))
stopwords1.add("I'm")
stopwords1.add("I've")
stopwords1.add('I')
stopwords1

{'I',
 "I'm",
 "I've",
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out

In [84]:
#Testing stopword removal

test = "the world is a beautiful place and positivity is the correct attitude"
test1 = test.split(' ')
t = [x for x in test1 if x not in stopwords1]
t

['world', 'beautiful', 'place', 'positivity', 'correct', 'attitude']

In [85]:
def cleanhtml(raw_html):
    '''
    Function to clean the data of hyperlinks and other text
    '''
    cleanr = re.compile('<.*?> ')
    cleanr1 = re.compile('<.*?>')
    cleantext1 = re.sub(cleanr, '', raw_html)
    cleantext = re.sub(cleanr1, '', cleantext1)
    return(cleantext)

In [86]:
def integer_frequency(list_str):
    '''
    Function to clean out line breaks and other not required text
    '''
    list_str = re.sub(r'^.*?https?:\/\/.*[\r\n]*', '', list_str, flags=re.MULTILINE)
    list_str = re.sub(r'^https?:\/\/.*[\r\n]*', '', list_str, flags=re.MULTILINE)
    list_str = list_str.replace('.', ' ')
    list_str = list_str.replace(',', ' ')
    list_str = list_str.replace('\n', ' ')
    list_str = list_str.replace('\r', ' ')
    list_str = list_str.replace('?', '')
    ls = [x for x in list_str.split(' ') if x not in stopwords1]
    return " ".join(ls)

### Testing the data cleaning

In [87]:
ques.Body.head(100)[24]

'<p>What are good libraries for C with datastructures like vectors, deques, stacks, hashmaps, treemaps, sets, etc.? Plain C, please, and platform-independent.</p>'

In [88]:
t = integer_frequency(cleanhtml(ques.Body.head(100)[24]))
t

'What good libraries C datastructures like vectors  deques  stacks  hashmaps  treemaps  sets  etc  Plain C  please  platform-independent '

In [89]:
s = "god I love love Piza Pizza you http://stackoverflow.com/questions/8050/beginners-guide-to-linq)"
integer_frequency(cleanhtml(s))

''

In [90]:
#Clean the whole dataset and combine Question Title & Body

ques['Body'] = ques['Body'].str.replace('<p>', '')
ques['Body'] = ques['Body'].str.replace('</p>', '')
ques['Body'] = ques['Body'].apply(lambda s: cleanhtml(s))
ques['Body'] = ques['Body'].apply(lambda s: integer_frequency(s))
ques['ques'] = ques['Title'] + ' ' + ques['Body']
ques_train = ques[['Id', 'ques']]
ques_train.head()
len(ques_train)

32604

In [91]:
r = list().append('vg')
print(r)

None


In [92]:
#Join all tags grouped by the question ID

tags = tags.groupby('Id').agg(lambda x: " ".join(x)).reset_index()
tags['Tag'] = tags['Tag'].apply(lambda x: x.split(" "))

In [101]:
#Merge the two datasets

ques_tags = pd.merge(left = ques_train, right = tags, how = 'left', on = 'Id')

In [102]:
ques_tags.head()

,Id,ques,Tag
0,80,SQLStatement.execute() - multiple queries in o...,"[flex, actionscript-3, air]"
1,90,Good branching and merging tutorials for Torto...,"[svn, tortoisesvn, branch, branching-and-merging]"
2,120,ASP.NET Site Maps Has anyone got experience cr...,"[sql, asp.net, sitemap]"
3,180,Function for creating color wheels This someth...,"[algorithm, language-agnostic, colors, color-s..."
4,260,Adding scripting functionality to .NET applica...,"[c#, .net, scripting, compiler-construction]"


In [19]:
#Examine the data

ques_tags.head(10)

,Id,ques,Tag
0,80,SQLStatement.execute() - multiple queries in o...,flex actionscript-3 air
1,90,Good branching and merging tutorials for Torto...,svn tortoisesvn branch branching-and-merging
2,120,ASP.NET Site Maps Has anyone got experience cr...,sql asp.net sitemap
3,180,Function for creating color wheels This someth...,algorithm language-agnostic colors color-space
4,260,Adding scripting functionality to .NET applica...,c# .net scripting compiler-construction
5,330,Should I use nested classes in this case? work...,c++ oop class nested-class
6,470,Homegrown consumption of web services writing ...,.net web-services
7,580,Deploying SQL Server Databases from Test to Li...,sql-server sql-server-2005 deployment release-...
8,650,Automatically update version number would like...,c# visual-studio versioning
9,810,Visual Studio Setup Project - Per User Registr...,windows visual-studio registry installation


In [20]:
len(ques_tags)

594057

### Final Cleaning

In [21]:
ques_tags['combined'] =ques_tags['Tag'].astype(str) + " " + ques_tags['ques'].astype(str)

In [22]:
ques_tags['combined'] = ques_tags['ques'].astype(str)
ques_tags['combined'] = ques_tags['combined'].str.replace('\r', '')
ques_tags['combined'] = ques_tags['combined'].str.replace('\n', '')

In [23]:
ques_tags.head()

,Id,ques,Tag,combined
0,80,SQLStatement.execute() - multiple queries in o...,flex actionscript-3 air,SQLStatement.execute() - multiple queries in o...
1,90,Good branching and merging tutorials for Torto...,svn tortoisesvn branch branching-and-merging,Good branching and merging tutorials for Torto...
2,120,ASP.NET Site Maps Has anyone got experience cr...,sql asp.net sitemap,ASP.NET Site Maps Has anyone got experience cr...
3,180,Function for creating color wheels This someth...,algorithm language-agnostic colors color-space,Function for creating color wheels This someth...
4,260,Adding scripting functionality to .NET applica...,c# .net scripting compiler-construction,Adding scripting functionality to .NET applica...


In [40]:
def get_noun_phrases(text):
    blob = TextBlob(text)
    return blob.noun_phrases

In [1]:
ques['ques_noun_phrases'] = ques['ques'].apply(lambda s: get_noun_phrases(s))

NameError: name 'ques' is not defined

In [44]:
ques['ques_noun_phrases'].head()

0    [sqlstatement.execute, multiple queries, datab...
1    [tortoisesvn, good tutorials, apache subversio...
2    [asp.net site maps, site-map providers, xml, m...
3    [function, color wheels, way generate colors, ...
4    [adding, .net applications, c #, uses database...
Name: ques_noun_phrases, dtype: object

In [24]:
def remove_stopwords(strn):
    return( " ".join([x for x in strn.split(" ") if x not in stopwords1]))

In [25]:
ques_tags['combined'] = ques_tags['combined'].apply(lambda s: remove_stopwords(s))

### Save to file

In [95]:
ques_tags[['Id', 'Tag', 'ques']].to_csv('clean_data_full.csv', sep='\t', encoding='utf-8')

In [104]:
ques_tags.head()

,Id,ques,Tag
0,80,SQLStatement.execute() - multiple queries in o...,"[flex, actionscript-3, air]"
1,90,Good branching and merging tutorials for Torto...,"[svn, tortoisesvn, branch, branching-and-merging]"
2,120,ASP.NET Site Maps Has anyone got experience cr...,"[sql, asp.net, sitemap]"
3,180,Function for creating color wheels This someth...,"[algorithm, language-agnostic, colors, color-s..."
4,260,Adding scripting functionality to .NET applica...,"[c#, .net, scripting, compiler-construction]"


In [139]:
X = ques_tags['ques'].values
y = ques_tags['Tag'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [142]:
print(y_train)
print(y_val)

[list(['c#', '.net', 'return-type'])
 list(['javascript', 'backbone.js', 'jsdoc'])
 list(['google-app-engine', 'amazon-web-services']) ...,
 list(['asp.net-mvc-3', 'razor', 'unobtrusive-javascript', 'unobtrusive-validation'])
 list(['c#', 'winforms', 'data-binding', 'datagridview', 'format'])
 list(['ios', 'swift', 'ios9', 'nsfetchedresultscontrolle'])]
[list(['javascript', 'jquery'])
 list(['c#', 'visual-studio-2010', 'web-deployment-project', 'web-platform-installer'])
 list(['c++', 'performance', 'simd', 'avx']) ...,
 list(['asp.net', 'asp.net-mvc'])
 list(['algorithm', 'sorting', 'radix-sort'])
 list(['javascript', 'html', 'html-table'])]


In [107]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def text_prepare(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replaces REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # deletes symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # deletes stopwors from text
    return text

In [108]:
X_train = [text_prepare(x) for x in X_train]
X_val = [text_prepare(x) for x in X_val]
X_test = [text_prepare(x) for x in X_test]

In [109]:
X_train[:3]

['return array collection function whats preferred container type returning multiple objects type function good practice return simple array like mytype wrap generic container like icollectionlt mytypegt thanks',
 'jsdoc annotate backbonejs code anyone ever documented backbonejs code jsdoc problems annotating backbone constructs user backbone model extend defaults 1 initialize function dosomething function p advice appreciated thanks',
 'google app engine vs amazon web services one developer startup service would pick goal get market really fast would go gae build everything scratch python one click deploy pick aws setup server db use rails admin work']

In [153]:
from collections import defaultdict
# Dictionary of all tags from train corpus with their counts.
tags_counts = defaultdict(int)
# Dictionary of all words from train corpus with their counts.
words_counts = defaultdict(int)

for tags in y_train:
    for tag in tags:
        tags_counts[tag] += 1

for tags in y_val:
    for tag in tags:
        if tag not in tags_counts:
            tags_counts[tag] += 1

for text in X_train:
    for word in text.split():
        words_counts[word] += 1

In [111]:
most_common_tags = sorted(tags_counts.items(), key=lambda x: x[1], reverse=True)[:3]
most_common_words = sorted(words_counts.items(), key=lambda x: x[1], reverse=True)[:3]

In [123]:
most_common_words

[('lt', 26727), ('0', 13688), ('1', 12934)]

In [114]:
DICT_SIZE = 5000
INDEX_TO_WORDS = sorted(words_counts.keys(), key=lambda x: words_counts[x], reverse=True)[:DICT_SIZE]
WORDS_TO_INDEX = {word:i for i, word in enumerate(INDEX_TO_WORDS)}
ALL_WORDS = WORDS_TO_INDEX.keys()

def my_bag_of_words(text, words_to_index, dict_size):
    """
        text: a string
        dict_size: size of the dictionary
        
        return a vector which is a bag-of-words representation of 'text'
    """
    text = text.split()
    result_vector = np.zeros(dict_size)
    for word in text:
        if word in words_to_index:
            result_vector[words_to_index[word]]+=1
    
    return result_vector

In [116]:
X_train_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_train])
X_val_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_val])
X_test_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_test])
print('X_train shape ', X_train_mybag.shape)
print('X_val shape ', X_val_mybag.shape)
print('X_test shape ', X_test_mybag.shape)

X_train shape  (20866, 5000)
X_val shape  (5217, 5000)
X_test shape  (6521, 5000)


In [118]:
def tfidf_features(X_train, X_val, X_test):
    """
        X_train, X_val, X_test — samples        
        return TF-IDF vectorized representation of each sample and vocabulary
    """        
    tfidf_vectorizer = TfidfVectorizer(min_df=5, max_df=0.9, ngram_range=(1, 2), token_pattern='(\S+)')
    X_train = tfidf_vectorizer.fit_transform(X_train)
    X_val = tfidf_vectorizer.transform(X_val)
    X_test = tfidf_vectorizer.transform(X_test)
    
    return X_train, X_val, X_test, tfidf_vectorizer.vocabulary_
X_train_tfidf, X_val_tfidf, X_test_tfidf, tfidf_vocab = tfidf_features(X_train, X_val, X_test)
tfidf_reversed_vocab = {i:word for word,i in tfidf_vocab.items()}

In [120]:
sorted(tfidf_vocab)[:10]

['#',
 '# #',
 '# +',
 '# 0',
 '# 1',
 '# 2',
 '# add',
 '# allow',
 '# bin',
 '# branch']

In [138]:
y_val

array([list(['javascript', 'jquery']),
       list(['c#', 'visual-studio-2010', 'web-deployment-project', 'web-platform-installer']),
       list(['c++', 'performance', 'simd', 'avx']), ...,
       list(['asp.net', 'asp.net-mvc']),
       list(['algorithm', 'sorting', 'radix-sort']),
       list(['javascript', 'html', 'html-table'])], dtype=object)

In [168]:
mlb = MultiLabelBinarizer(classes=sorted(tags_counts.keys()))
mlb1 = MultiLabelBinarizer(classes=sorted(tags_counts.keys()))
#y_train = mlb.fit_transform(y_train)

#y_val = mlb.fit_transform(y_val)

In [169]:
y_train_1 = mlb.fit_transform(y_train)

In [156]:
y_val

array([list(['javascript', 'jquery']),
       list(['c#', 'visual-studio-2010', 'web-deployment-project', 'web-platform-installer']),
       list(['c++', 'performance', 'simd', 'avx']), ...,
       list(['asp.net', 'asp.net-mvc']),
       list(['algorithm', 'sorting', 'radix-sort']),
       list(['javascript', 'html', 'html-table'])], dtype=object)

In [170]:
y_val_1 = mlb1.fit_transform(y_val)

In [171]:

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition.nmf import NMF
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier


In [172]:
#tried several classifiers and picked the best one.
def train_classifier(X_train, y_train):
    """
      X_train, y_train — training data
      
      return: trained classifier
    """
    
#     clf = OneVsRestClassifier(MLPClassifier(hidden_layer_sizes=(16,8,8), learning_rate='adaptive', max_iter=2000,verbose=True))
#     clf = OneVsRestClassifier(SVC(max_iter=1000, verbose=True))
#     clf = OneVsRestClassifier(LogisticRegression())
#    clf = OneVsRestClassifier(AdaBoostClassifier())
    clf = OneVsRestClassifier(RidgeClassifier(normalize=True))
    clf.fit(X_train, y_train)
    return clf   

In [ ]:
classifier_mybag = train_classifier(X_train_mybag, y_train_1)
classifier_tfidf = train_classifier(X_train_tfidf, y_train_1)

C:\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 21 is present in all training examples.
  str(classes[c]))
C:\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 23 is present in all training examples.
  str(classes[c]))
C:\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 26 is present in all training examples.
  str(classes[c]))
C:\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 32 is present in all training examples.
  str(classes[c]))
C:\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 36 is present in all training examples.
  str(classes[c]))
C:\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 39 is present in all training examples.
  str(classes[c]))
C:\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 62 is present in all training examples.
  str(classes[c]))
C:\Anaconda3\lib\site-packa

C:\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 527 is present in all training examples.
  str(classes[c]))
C:\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 546 is present in all training examples.
  str(classes[c]))
C:\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 579 is present in all training examples.
  str(classes[c]))
C:\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 589 is present in all training examples.
  str(classes[c]))
C:\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 601 is present in all training examples.
  str(classes[c]))
C:\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 613 is present in all training examples.
  str(classes[c]))
C:\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 615 is present in all training examples.
  str(classes[c]))


In [ ]:
y_val_predicted_labels_mybag = classifier_mybag.predict(X_val_mybag)
y_val_predicted_scores_mybag = classifier_mybag.decision_function(X_val_mybag)

y_val_predicted_labels_tfidf = classifier_tfidf.predict(X_val_tfidf)
y_val_predicted_scores_tfidf = classifier_tfidf.decision_function(X_val_tfidf)

In [ ]:
y_val_pred_inversed = mlb.inverse_transform(y_val_predicted_labels_tfidf)
y_val_inversed = mlb.inverse_transform(y_val_1)
for i in range(6,12):
    print('Title:\t{}\nTrue labels:\t{}\nPredicted labels:\t{}\n\n'.format(
        X_val[i],
        ','.join(y_val_inversed[i]),
        ','.join(y_val_pred_inversed[i])
    ))

### Data Verification (Rough Work)



In [29]:
re.findall(r"[\w']+", 'yo yo honey signh')

['yo', 'yo', 'honey', 'signh']

In [30]:
data = pd.read_csv('./clean_data_full.csv', sep = '\t')

In [101]:
len(data)

594057

In [134]:
data.head()

,Unnamed: 0,Id,Tag,combined
0,0,80,flex actionscript-3 air,SQLStatement.execute() - multiple queries one ...
1,1,90,svn tortoisesvn branch branching-and-merging,Good branching merging tutorials TortoiseSVN? ...
2,2,120,sql asp.net sitemap,ASP.NET Site Maps Has anyone got experience cr...
3,3,180,algorithm language-agnostic colors color-space,Function creating color wheels This something ...
4,4,260,c# .net scripting compiler-construction,Adding scripting functionality .NET applicatio...


In [64]:
x = ['agg', 'afaef', 'agaga']

In [68]:
x + ['khgb']

['agg', 'afaef', 'agaga', 'khgb']

In [135]:
sentences= []
for elem in data.head(10).iterrows():
    tup = elem[1]
    x = [str(tup['Id'])]
    y = tup['Tag'].split(" ")
    tags = x + y
    a = str(tup['combined']).split(' ')
    t = [x.replace('}', '').replace('(', '').replace('.', ' ').replace(')', '').replace('{', '').replace(';', '').replace('-', '').replace('_', ' ') for x in a if(x != '' and len(x) > 2)]
    sentences.append([t, tags])
    
sentences

[[['SQLStatement execute',
   'multiple',
   'queries',
   'one',
   'statement',
   'written',
   'database',
   'generation',
   'script',
   'want',
   'execute',
   'application:',
   'Create',
   'Table',
   'tRole',
   'roleID',
   'integer',
   'Primary',
   'Key',
   'roleName',
   'varchar40',
   'Create',
   'Table',
   'tFile',
   'fileID',
   'integer',
   'Primary',
   'Key',
   'fileName',
   'varchar50',
   'fileDescription',
   'varchar500',
   'thumbnailID',
   'integer',
   'fileFormatID',
   'integer',
   'categoryID',
   'integer',
   'isFavorite',
   'boolean',
   'dateAdded',
   'date',
   'globalAccessCount',
   'integer',
   'lastAccessTime',
   'date',
   'downloadComplete',
   'boolean',
   'isNew',
   'boolean',
   'isSpotlight',
   'boolean',
   'duration',
   'varchar30',
   'Create',
   'Table',
   'tCategory',
   'categoryID',
   'integer',
   'Primary',
   'Key',
   'categoryName',
   'varchar50',
   'parent categoryID',
   'integer',
   'execute',
   'A

In [27]:
for elem in data.head().iterrows():
    tup = elem[1]
    print(tup['Tag'])

flex actionscript-3 air
svn tortoisesvn branch branching-and-merging
sql asp.net sitemap
algorithm language-agnostic colors color-space
c# .net scripting compiler-construction


In [ ]:
for elem in data.head().iterrows():
    tup = elem[1]
    print(tup['Id'])

In [87]:
for elem in data.head(60).iterrows():
    tup = elem[1]
    print(tup['combined'].split(' '))

['flex', 'actionscript-3', 'air', 'SQLStatement.execute()', '-', 'multiple', 'queries', 'one', 'statement', 'integer']
['svn', 'tortoisesvn', 'branch', 'branching-and-merging', 'Good', 'branching', 'merging', 'tutorials', 'TortoiseSVN?', 'TortoiseSVN', 'explaining', 'Subversion', 'tutorials', 'specific', 'really', 'Apache', 'better', 'client', 'good', "it's", 'Are', 'All']
['sql', 'asp.net', 'sitemap', 'ASP.NET', 'Site', 'Maps', 'dynamically', 'permissions', 'experience', 'membership', 'providers', 'creating', 'site-map', 'properly', 'controls', 'need', 'standard', 'default', 'working', 'viewing', 'anyone', 'got', 'create', 'modify', 'system', 'users', 'pages', "I've", 'file', 'Menu', "I'll", 'site', 'page', 'well', 'Has', 'XML', 'way', 'tie']
['algorithm', 'language-agnostic', 'colors', 'color-space', 'Function', 'creating', 'color', 'wheels', 'distinguishable', 'pseudo-solved', 'something', 'parameter', 'solution', 'generate', 'possible', 'problem', "That's", 'colors']
['c#', '.net',

In [90]:
data[data['Id'].astype(str) == '1272130']['combined'].item()

'java file permissions Checking write access directory creating files inside directory'

In [88]:
" ".join(['sql', 'asp.net', 'sitemap', 'ASP.NET', 'Site', 'Maps', 'dynamically', 'permissions', 'experience', 'membership', 'providers', 'creating', 'site-map', 'properly', 'controls', 'need', 'standard', 'default', 'working', 'viewing', 'anyone', 'got', 'create', 'modify', 'system', 'users', 'pages', "I've", 'file', 'Menu', "I'll", 'site', 'page', 'well', 'Has', 'XML', 'way', 'tie'])

"sql asp.net sitemap ASP.NET Site Maps dynamically permissions experience membership providers creating site-map properly controls need standard default working viewing anyone got create modify system users pages I've file Menu I'll site page well Has XML way tie"